# IFN695 - Report Code

Importing all relevant libraries

In [343]:
import pandas as pd
import numpy as np
from nemosis import dynamic_data_compiler
import plotly.express as px
from datetime import datetime

pd.set_option('display.max_rows', 20)

Importing the Renewable Penetration Dataset

In [3]:
renewable_penetration = pd.read_csv('data/NEM_RENEWABLE_PENETRATION_ALL_202508041827.csv')
renewable_penetration

,DateTime,Max/Min,State,Fuel Type,Supply
0,2/1/2018 4:00,Min,NEM,Battery,0.000
1,2/1/2018 4:00,Min,NEM,Biomass,10.233
2,2/1/2018 4:00,Min,NEM,Black coal,10655.808
3,2/1/2018 4:00,Min,NEM,Brown coal,4655.597
4,2/1/2018 4:00,Min,NEM,Distributed PV,0.000
...,...,...,...,...,...
1858,3/8/2025 11:30,Max,NEM,Gas,258.971
1859,3/8/2025 11:30,Max,NEM,Hydro,586.385
1860,3/8/2025 11:30,Max,NEM,Liquid Fuel,0.000
1861,3/8/2025 11:30,Max,NEM,Utility-scale Solar,5725.921


Loading the Archived Packages for Fuel Mix in Each State

In [388]:
start_date = '2024/08/01 00:00:00'
end_date = '2024/08/01 00:00:00'

raw_data_cache = '/Users/aidanlockwood/Documents/GitHub/IFN695-Codebase/data/fuel_mix'

dudetail = dynamic_data_compiler(
    start_time = start_date,
    end_time = end_date,
    raw_data_location = raw_data_cache, 
    table_name = 'DUDETAILSUMMARY'
)

dispatch = dynamic_data_compiler(
    start_time = start_date,
    end_time = end_date,
    raw_data_location = raw_data_cache,
    table_name = 'DISPATCH_UNIT_SCADA'
)


INFO: Compiling data for table DUDETAILSUMMARY
INFO: Returning DUDETAILSUMMARY.
INFO: Compiling data for table DISPATCH_UNIT_SCADA
INFO: Downloading data for table DISPATCH_UNIT_SCADA, year 2024, month 07
INFO: Creating feather file for DISPATCH_UNIT_SCADA, 2024, 07
INFO: Downloading data for table DISPATCH_UNIT_SCADA, year 2024, month 08
INFO: Creating feather file for DISPATCH_UNIT_SCADA, 2024, 08
INFO: Returning DISPATCH_UNIT_SCADA.


In [387]:
dudetail_data = pd.read_csv('data/fuel_mix/PUBLIC_DVD_DUDETAILSUMMARY_202407010000.CSV', header = 1)
dudetail_generator_data = dudetail_data[dudetail_data['DISPATCHTYPE'] == 'GENERATOR']
dudetail_generator_data

,I,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7,DUID,START_DATE,END_DATE,DISPATCHTYPE,CONNECTIONPOINTID,REGIONID,...,IS_AGGREGATED,DISPATCHSUBTYPE,ADG_ID,LOAD_MINIMUM_ENERGY_PRICE,LOAD_MAXIMUM_ENERGY_PRICE,LOAD_MIN_RAMP_RATE_UP,LOAD_MIN_RAMP_RATE_DOWN,LOAD_MAX_RAMP_RATE_UP,LOAD_MAX_RAMP_RATE_DOWN,SECONDARY_TLF
0,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,ADPBA1G,2021/04/20 00:00:00,2021/06/29 00:00:00,GENERATOR,SMVE4D,SA1,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,ADPBA1G,2021/06/29 00:00:00,2021/07/01 00:00:00,GENERATOR,SMVE4D,SA1,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,ADPBA1G,2021/07/01 00:00:00,2021/11/09 00:00:00,GENERATOR,SMVE4D,SA1,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,ADPBA1G,2021/11/09 00:00:00,2022/07/01 00:00:00,GENERATOR,SMVE4D,SA1,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,ADPBA1G,2022/07/01 00:00:00,2023/07/01 00:00:00,GENERATOR,SMVE4D,SA1,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13223,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,YWPS4,2020/07/01 00:00:00,2021/07/01 00:00:00,GENERATOR,VYP24,VIC1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13224,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,YWPS4,2021/07/01 00:00:00,2022/07/01 00:00:00,GENERATOR,VYP24,VIC1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13225,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,YWPS4,2022/07/01 00:00:00,2023/07/01 00:00:00,GENERATOR,VYP24,VIC1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13226,D,PARTICIPANT_REGISTRATION,DUDETAILSUMMARY,7.0,YWPS4,2023/07/01 00:00:00,2024/07/01 00:00:00,GENERATOR,VYP24,VIC1,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [392]:
dispatch_august_24 = pd.read_csv('data/fuel_mix/PUBLIC_ARCHIVE#DISPATCH_UNIT_SCADA#FILE01#202408010000.CSV', header = 1)
dispatch_august_24

,I,DISPATCH,UNIT_SCADA,1,SETTLEMENTDATE,DUID,SCADAVALUE
0,D,DISPATCH,UNIT_SCADA,1.0,2024/08/01 00:05:00,ADPBA1G,0.00000
1,D,DISPATCH,UNIT_SCADA,1.0,2024/08/01 00:05:00,ADPBA1L,2.90700
2,D,DISPATCH,UNIT_SCADA,1.0,2024/08/01 00:05:00,ADPPV1,0.00000
3,D,DISPATCH,UNIT_SCADA,1.0,2024/08/01 00:05:00,AGLHAL,0.00000
4,D,DISPATCH,UNIT_SCADA,1.0,2024/08/01 00:05:00,AGLSOM,19.82500
...,...,...,...,...,...,...,...
4136173,D,DISPATCH,UNIT_SCADA,1.0,2024/09/01 00:00:00,YWPS1,248.36710
4136174,D,DISPATCH,UNIT_SCADA,1.0,2024/09/01 00:00:00,YWPS2,274.18228
4136175,D,DISPATCH,UNIT_SCADA,1.0,2024/09/01 00:00:00,YWPS3,217.12025
4136176,D,DISPATCH,UNIT_SCADA,1.0,2024/09/01 00:00:00,YWPS4,212.32152


### Need to Connect the DUID to the Region ID

In [370]:
duid_regions = pd.DataFrame({
    'DUID' : dudetail_data['DUID'],
    'REGIONID' : dudetail_data['REGIONID']
})
duid_regions

,DUID,REGIONID
0,ADPBA1G,SA1
1,ADPBA1G,SA1
2,ADPBA1G,SA1
3,ADPBA1G,SA1
4,ADPBA1G,SA1
...,...,...
13224,YWPS4,VIC1
13225,YWPS4,VIC1
13226,YWPS4,VIC1
13227,YWPS4,VIC1


In [382]:
duid_regions['DUID'].unique()

array(['ADPBA1G', 'ADPBA1L', 'ADPMH1', 'ADPPV1', 'ADPPV2', 'ADPPV3',
       'AGLHAL', 'AGLNOW1', 'AGLSITA1', 'AGLSOM', 'AMCORGR', 'ANGAS1',
       'ANGAS2', 'ANGAST1', 'APD01', 'APD02', 'APPIN', 'APS', 'ARWF1',
       'ASENGV01', 'ASENGY01', 'ASNACTW1', 'ASNAES1', 'ASNBRL1',
       'ASNENC1', 'ASNSEL2', 'ASPAPM01', 'ASPSTV01', 'ASQEEV1', 'ASQENC1',
       'ASRMGE01', 'ASRMGE02', 'ASRMGE03', 'ASRMGE04', 'ASSDE1', 'ASSEL1',
       'ASSENC1', 'ASSERDV1', 'ASSSN1', 'ASTFG1V1', 'ASTHYD1', 'ASVIEL2',
       'AS_NSW1', 'AVLSF1', 'AWABAREF', 'BAKING1', 'BALB1', 'BALBG1',
       'BALBL1', 'BALDHWF1', 'BANGOWF1', 'BANGOWF2', 'BANKSPT1', 'BANN1',
       'BAPS', 'BARCALDN', 'BARCSF1', 'BARKIPS1', 'BARRON-1', 'BARRON-2',
       'BASTYAN', 'BAYNL1', 'BBASEHOS', 'BBATRYL1', 'BBATTERY',
       'BBDISEL1', 'BBTHREE1', 'BBTHREE2', 'BBTHREE3', 'BDL01', 'BDL02',
       'BDONGHYD', 'BELLBAY1', 'BELLBAY2', 'BERWICK', 'BERYLSF1',
       'BHASZINC', 'BHB1', 'BHBG1', 'BHBL1', 'BLNKTAS', 'BLNKVIC',
       'BLOW

In [4]:
renewable_penetration.dtypes

DateTime      object
Max/Min       object
State         object
Fuel Type     object
Supply       float64
dtype: object

Having a look at the different fuel types in the dataset

In [8]:
renewable_penetration['Fuel Type'].value_counts()

Fuel Type
Battery                184
Biomass                184
Black coal             184
Brown coal             184
Distributed PV         184
Gas                    184
Hydro                  184
Liquid Fuel            184
Utility-scale Solar    184
Wind                   184
Other                   23
Name: count, dtype: int64

There is an 'Other' value for Fuel Type. Going to remove that for easier analysis

In [10]:
renewable_penetration = renewable_penetration[renewable_penetration['Fuel Type'] != 'Other']
renewable_penetration

,DateTime,Max/Min,State,Fuel Type,Supply
0,2018-01,Min,NEM,Battery,0.000
1,2018-01,Min,NEM,Biomass,10.233
2,2018-01,Min,NEM,Black coal,10655.808
3,2018-01,Min,NEM,Brown coal,4655.597
4,2018-01,Min,NEM,Distributed PV,0.000
...,...,...,...,...,...
1858,2025-08,Max,NEM,Gas,258.971
1859,2025-08,Max,NEM,Hydro,586.385
1860,2025-08,Max,NEM,Liquid Fuel,0.000
1861,2025-08,Max,NEM,Utility-scale Solar,5725.921


The Supply column is the only column that has the correct format. Going to change the:
- DateTime to a Date Object - as this will be a monthly average, min and max 
- FuelType to a category 
- Max/Min transposed into their own variables with a calculated monthly average

Converting the Date Column

In [11]:
# Converting the DateTime column
renewable_penetration['DateTime'] = pd.to_datetime(renewable_penetration['DateTime'], format='mixed', dayfirst=True)
renewable_penetration['DateTime'] = renewable_penetration['DateTime'].dt.strftime('%Y-%m')

### Creating the Groupby's to Calculate the Min, Max and Mean Supply Amounts

In [63]:
mean_supply = renewable_penetration.groupby(['Fuel Type', 'DateTime'])['Supply'].mean().reset_index()
min_supply = renewable_penetration.groupby(['Fuel Type', 'DateTime'])['Supply'].min().reset_index()['Supply']
max_supply = renewable_penetration.groupby(['Fuel Type', 'DateTime'])['Supply'].max().reset_index()['Supply']

# Adding the min and max supplies to the mean supply groupby
mean_supply['Min Supply (MW)'] = min_supply
mean_supply['Max Supply (MW)'] = max_supply

In [65]:
mean_supply.columns = ['Fuel Type', 'Month', 'Mean Supply (MW)', 'Min Supply (MW)', 'Max Supply (MW)']
mean_supply

,Fuel Type,Month,Mean Supply (MW),Min Supply (MW),Max Supply (MW)
0,Battery,2018-01,0.2830,0.000,0.566
1,Battery,2018-02,0.6665,0.000,1.333
2,Battery,2018-03,2.8665,0.000,5.733
3,Battery,2018-04,4.8250,0.000,9.650
4,Battery,2018-05,3.0330,2.550,3.516
...,...,...,...,...,...
915,Wind,2025-04,2027.1265,1279.964,2774.289
916,Wind,2025-05,3007.9465,1232.864,4783.029
917,Wind,2025-06,3260.6030,1857.128,4664.078
918,Wind,2025-07,3395.3705,1422.093,5368.648


Generating the new dataframe with the supplies, sorted by months

In [68]:
supply_summary = mean_supply.sort_values(by = 'Month').reset_index(drop = True)
supply_summary

,Fuel Type,Month,Mean Supply (MW),Min Supply (MW),Max Supply (MW)
0,Battery,2018-01,0.2830,0.000,0.566
1,Liquid Fuel,2018-01,0.0595,0.021,0.098
2,Black coal,2018-01,12383.7065,10655.808,14111.605
3,Hydro,2018-01,1122.2300,355.826,1888.634
4,Biomass,2018-01,12.3745,10.233,14.516
...,...,...,...,...,...
915,Black coal,2025-08,9114.7900,6399.196,11830.384
916,Biomass,2025-08,40.5620,35.100,46.024
917,Battery,2025-08,7.2425,2.324,12.161
918,Utility-scale Solar,2025-08,2863.1915,0.462,5725.921


In [72]:
battery_summary = supply_summary[supply_summary['Fuel Type'] == 'Battery']

In [76]:
supply_summary[supply_summary['Month'] == '2018-01']

,Fuel Type,Month,Mean Supply (MW),Min Supply (MW),Max Supply (MW)
0,Battery,2018-01,0.2830,0.000,0.566
1,Liquid Fuel,2018-01,0.0595,0.021,0.098
2,Black coal,2018-01,12383.7065,10655.808,14111.605
3,Hydro,2018-01,1122.2300,355.826,1888.634
4,Biomass,2018-01,12.3745,10.233,14.516
5,Wind,2018-01,1715.5270,566.570,2864.484
6,Brown coal,2018-01,4597.9545,4540.312,4655.597
7,Gas,2018-01,1712.7820,1517.622,1907.942
8,Distributed PV,2018-01,1938.2870,0.000,3876.574
9,Utility-scale Solar,2018-01,134.7040,2.204,267.204


In [243]:
supply_summary['Fuel Type'].unique()

array(['Battery', 'Wind', 'Utility-scale Solar', 'Liquid Fuel', 'Hydro',
       'Distributed PV', 'Brown coal', 'Black coal', 'Biomass', 'Gas'],
      dtype=object)

In [80]:
mean_supply_amounts = supply_summary.groupby(['Month'])['Mean Supply (MW)'].sum().reset_index()['Mean Supply (MW)']

### Creating a DataFrame to split Renewables and Non-Renewables

In [263]:
non_renewables = ['Liquid Fuel', 'Brown coal', 'Black coal', 'Gas']

renewables = supply_summary['Fuel Type'].unique().tolist()

for fuel in non_renewables:
    renewables.remove(fuel)

renewables_df = supply_summary[supply_summary['Fuel Type'].isin(renewables)].reset_index(drop = True)
non_renewables_df = supply_summary[supply_summary['Fuel Type'].isin(non_renewables)].reset_index(drop = True)

In [264]:
renewables_df

,Fuel Type,Month,Mean Supply (MW),Min Supply (MW),Max Supply (MW)
0,Battery,2018-01,0.2830,0.000,0.566
1,Wind,2018-01,1715.5270,566.570,2864.484
2,Utility-scale Solar,2018-01,134.7040,2.204,267.204
3,Hydro,2018-01,1122.2300,355.826,1888.634
4,Distributed PV,2018-01,1938.2870,0.000,3876.574
...,...,...,...,...,...
546,Battery,2025-08,7.2425,2.324,12.161
547,Utility-scale Solar,2025-08,2863.1915,0.462,5725.921
548,Hydro,2025-08,1224.0985,586.385,1861.812
549,Distributed PV,2025-08,5037.2530,0.000,10074.506


In [ ]:
renewables_df[0:7]

,Fuel Type,Month,Mean Supply (MW),Min Supply (MW),Max Supply (MW)
0,Battery,2018-01,0.2830,0.000,0.566
1,Wind,2018-01,1715.5270,566.570,2864.484
2,Utility-scale Solar,2018-01,134.7040,2.204,267.204
3,Hydro,2018-01,1122.2300,355.826,1888.634
4,Distributed PV,2018-01,1938.2870,0.000,3876.574
5,Biomass,2018-01,12.3745,10.233,14.516
6,Hydro,2018-02,554.1920,554.192,554.192


### Creating a DataFrame to Aggregate the Mean Supplies of all Fuel Sources
Then plot the total mean supply over time

In [270]:
renewables_supply = renewables_df.groupby(['Month'])['Mean Supply (MW)'].sum().reset_index()
renewables_supply

,Month,Mean Supply (MW)
0,2018-01,4923.4055
1,2018-02,995.1190
2,2018-03,4460.9820
3,2018-04,4512.6620
4,2018-05,4857.1565
...,...,...
87,2025-04,10536.8795
88,2025-05,10704.8115
89,2025-06,11253.8470
90,2025-07,12593.5200


In [272]:
non_renewables_supply = non_renewables_df.groupby(['Month'])['Mean Supply (MW)'].sum().reset_index()
non_renewables_supply

,Month,Mean Supply (MW)
0,2018-01,18694.5025
1,2018-02,18519.9090
2,2018-03,17066.9565
3,2018-04,16415.0150
4,2018-05,17820.6030
...,...,...
87,2025-04,12175.0755
88,2025-05,12536.7005
89,2025-06,13385.9000
90,2025-07,15253.6980


In [275]:
total_supply_df = pd.merge(renewables_supply, non_renewables_supply, on='Month', suffixes=(' Renewables', ' Non Renewables'))
total_supply_df

,Month,Mean Supply (MW) Renewables,Mean Supply (MW) Non Renewables
0,2018-01,4923.4055,18694.5025
1,2018-02,995.1190,18519.9090
2,2018-03,4460.9820,17066.9565
3,2018-04,4512.6620,16415.0150
4,2018-05,4857.1565,17820.6030
...,...,...,...
87,2025-04,10536.8795,12175.0755
88,2025-05,10704.8115,12536.7005
89,2025-06,11253.8470,13385.9000
90,2025-07,12593.5200,15253.6980


### Generating the Plots for the Total Fuel Source Renewables/Non-Renewables

In [281]:
fig = px.line(total_supply_df, x='Month', y=['Mean Supply (MW) Renewables', 'Mean Supply (MW) Non Renewables'], title='Total Mean Supply of Non-Renewables vs Renewables in the NEM (2018 - 2025)', labels={'value': 'Mean Supply (MW)', 'variable': 'Fuel Type'})
fig.update_traces(mode = 'lines+markers')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': ('Fuel Type=Mean Supply (MW) Ren' ... 'upply (MW)=%{y}<extra></extra>'),
              'legendgroup': 'Mean Supply (MW) Renewables',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': 'Mean Supply (MW) Renewables',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                          '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
                          '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06',
                          '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
                          '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
                          '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
                          '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06',
                          '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12',
                          '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
                          '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
                          '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06',
                          '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12',
                          '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
                          '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12',
                          '2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06',
                          '2025-07', '2025-08'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('hxbZzmc7s0DLoUW28xiPQKwcWmT7bL' ... 'rs+sVA9ihcj8KYyECe76fGa6bHQA=='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': ('Fuel Type=Mean Supply (MW) Non' ... 'upply (MW)=%{y}<extra></extra>'),
              'legendgroup': 'Mean Supply (MW) Non Renewables',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines+markers',
              'name': 'Mean Supply (MW) Non Renewables',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                          '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
                          '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06',
                          '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
                          '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
                          '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
                          '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06',
                          '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12',
                          '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
                          '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
                          '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06',
                          '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12',
                          '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
                          '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12',
                          '2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06',
                          '2025-07',

### Generating the Plot for Renewables

In [285]:
px.line(renewables_df, x = 'Month', y = 'Mean Supply (MW)', color = 'Fuel Type', title = 'Mean Monthly Supply of Renewable Energy Sources (2018-2025)')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Fuel Type=Battery<br>Month=%{x}<br>Mean Supply (MW)=%{y}<extra></extra>',
              'legendgroup': 'Battery',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Battery',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                          '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
                          '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06',
                          '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
                          '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
                          '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
                          '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06',
                          '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12',
                          '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
                          '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
                          '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06',
                          '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12',
                          '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
                          '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12',
                          '2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06',
                          '2025-07', '2025-08'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('6SYxCKwc0j+HFtnO91P1PzvfT42X7g' ... 'tsxypAYhBYObSoNkC4HoXrUfgcQA=='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'Fuel Type=Wind<br>Month=%{x}<br>Mean Supply (MW)=%{y}<extra></extra>',
              'legendgroup': 'Wind',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Wind',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                          '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
                          '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06',
                          '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
                          '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
                          '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
                          '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06',
                          '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12',
                          '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
                          '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
                          '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06',
                          '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12',
                          '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
                          '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12',
                          '2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06',
                          '2025-07', '2025-08'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('+FPjpRvOmkDLoUW28197QN

### Generating the Plot for Non-Renewables

In [287]:
px.line(non_renewables_df, x = 'Month', y = 'Mean Supply (MW)', color = 'Fuel Type', title = 'Mean Monthly Supply of Non-Renewable Energy Sources (2018-2025)')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Fuel Type=Liquid Fuel<br>Month=%{x}<br>Mean Supply (MW)=%{y}<extra></extra>',
              'legendgroup': 'Liquid Fuel',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Liquid Fuel',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                          '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
                          '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06',
                          '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
                          '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
                          '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
                          '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06',
                          '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12',
                          '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
                          '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
                          '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06',
                          '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12',
                          '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
                          '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12',
                          '2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06',
                          '2025-07', '2025-08'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('EVg5tMh2rj+LbOf7qfGiP7pJDAIrh6' ... 'AAAAAAf2q8dJNYOUAAAAAAAAAAAA=='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'Fuel Type=Brown coal<br>Month=%{x}<br>Mean Supply (MW)=%{y}<extra></extra>',
              'legendgroup': 'Brown coal',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Brown coal',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                          '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
                          '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06',
                          '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
                          '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
                          '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
                          '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06',
                          '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12',
                          '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
                          '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
                          '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06',
                          '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12',
                          '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
                          '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12',
                          '2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06',
                          '2025-07', '2025-08'], dtype=object),
              'xaxis': 'x',
              'y': {'bd

### Getting the Mean Supplies for Each Renewable Fuel Source in October 2023 and September 2024

In [297]:
october_2023 = renewables_df[renewables_df['Month'] == '2023-10'].sort_values(by = 'Fuel Type').reset_index(drop=True)
october_2023

,Fuel Type,Month,Mean Supply (MW),Min Supply (MW),Max Supply (MW)
0,Battery,2023-10,18.4530,9.029,27.877
1,Biomass,2023-10,66.5195,62.199,70.840
2,Distributed PV,2023-10,5842.3330,0.000,11684.666
3,Hydro,2023-10,1125.4695,445.747,1805.192
4,Utility-scale Solar,2023-10,2356.6765,0.620,4712.733
5,Wind,2023-10,2514.8595,1579.012,3450.707


In [298]:
september_2024 = renewables_df[renewables_df['Month'] == '2024-09'].sort_values(by = 'Fuel Type').reset_index(drop=True)
september_2024

,Fuel Type,Month,Mean Supply (MW),Min Supply (MW),Max Supply (MW)
0,Battery,2024-09,-83.5645,-145.975,-21.154
1,Biomass,2024-09,80.9705,76.527,85.414
2,Distributed PV,2024-09,5417.1240,0.000,10834.248
3,Hydro,2024-09,947.0640,481.970,1412.158
4,Utility-scale Solar,2024-09,2582.4230,0.027,5164.819
5,Wind,2024-09,2992.2475,2204.795,3779.700


In [299]:
october_september_mean_supplies = pd.DataFrame({
    'Fuel Type': october_2023['Fuel Type'],
    'Mean Supply October 2023 (MW)': october_2023['Mean Supply (MW)'],
    'Mean Supply September 2024 (MW)': september_2024['Mean Supply (MW)']
})
october_september_mean_supplies

,Fuel Type,Mean Supply October 2023 (MW),Mean Supply September 2024 (MW)
0,Battery,18.4530,-83.5645
1,Biomass,66.5195,80.9705
2,Distributed PV,5842.3330,5417.1240
3,Hydro,1125.4695,947.0640
4,Utility-scale Solar,2356.6765,2582.4230
5,Wind,2514.8595,2992.2475


### Producing the Minimum Negative Supply Amounts and their Frequency

In [312]:
min_neg_supply = supply_summary[supply_summary['Min Supply (MW)'] < 0].groupby('Fuel Type').min().drop(columns = ['Mean Supply (MW)', 'Max Supply (MW)'])

In [314]:
min_neg_freq = supply_summary[supply_summary['Min Supply (MW)'] < 0]['Fuel Type'].value_counts().to_list()
min_neg_supply['Frequency of Negative Supply Months'] = min_neg_freq
min_neg_supply

,Month,Min Supply (MW),Frequency of Negative Supply Months
Fuel Type,,,
Battery,2024-05,-161.050,15
Biomass,2022-10,-4.950,6
Liquid Fuel,2022-10,-0.100,5
Utility-scale Solar,2022-08,-0.202,4


Cell used to export the data to Excel

In [315]:
min_neg_supply.to_excel('data/min_negative_supplies.xlsx')

### Function to Produce DataFrame for the CAGR of each Source

In [321]:
renewables_df

,Fuel Type,Month,Mean Supply (MW),Min Supply (MW),Max Supply (MW)
0,Battery,2018-01,0.2830,0.000,0.566
1,Wind,2018-01,1715.5270,566.570,2864.484
2,Utility-scale Solar,2018-01,134.7040,2.204,267.204
3,Hydro,2018-01,1122.2300,355.826,1888.634
4,Distributed PV,2018-01,1938.2870,0.000,3876.574
...,...,...,...,...,...
546,Battery,2025-08,7.2425,2.324,12.161
547,Utility-scale Solar,2025-08,2863.1915,0.462,5725.921
548,Hydro,2025-08,1224.0985,586.385,1861.812
549,Distributed PV,2025-08,5037.2530,0.000,10074.506


In [330]:
def produce_cagr_dataframe(data, start_date, end_date, num_years = 7):

    fuel_sources = data['Fuel Type'].unique().tolist()

    fuel_data = {}

    print(fuel_sources)

    for fuel in fuel_sources:
        start_mean_supply = data[(data['Fuel Type'] == fuel) &(data['Month'] == start_date)]['Mean Supply (MW)'].values[0]
        end_mean_supply = data[(data['Fuel Type'] == fuel) & (data['Month'] == end_date)]['Mean Supply (MW)'].values[0]

        cagr = (((end_mean_supply / start_mean_supply) ** (1 / num_years)) - 1) * 100

        fuel_data[fuel] = [cagr, start_mean_supply, end_mean_supply]

    return pd.DataFrame(fuel_data, index = ['CAGR (%)', f'Start Mean Supply (MW)', f'End Mean Supply (MW)']).T

In [332]:
renewables_cagr = produce_cagr_dataframe(renewables_df, '2018-01', '2025-08')
renewables_cagr

['Battery', 'Wind', 'Utility-scale Solar', 'Hydro', 'Distributed PV', 'Biomass']


,CAGR (%),Start Mean Supply (MW),End Mean Supply (MW)
Battery,58.912273,0.2830,7.2425
Wind,7.981008,1715.5270,2936.4945
Utility-scale Solar,54.752815,134.7040,2863.1915
Hydro,1.248976,1122.2300,1224.0985
Distributed PV,14.618222,1938.2870,5037.2530
Biomass,18.482977,12.3745,40.5620


In [333]:
non_renewables_cagr = produce_cagr_dataframe(non_renewables_df, '2018-01', '2025-08')
non_renewables_cagr

['Liquid Fuel', 'Brown coal', 'Black coal', 'Gas']


,CAGR (%),Start Mean Supply (MW),End Mean Supply (MW)
Liquid Fuel,-100.000000,0.0595,0.0000
Brown coal,-3.237220,4597.9545,3651.9350
Black coal,-4.283867,12383.7065,9114.7900
Gas,-8.579342,1712.7820,914.1395


Writing the data to an Excel File

In [334]:
renewables_cagr.to_excel('data/renewables_cagr.xlsx')
non_renewables_cagr.to_excel('data/non_renewables_cagr.xlsx')